In [4]:
import requests
import re
from bs4 import BeautifulSoup
import os
import time
import pandas as pd

In [ ]:
url = 'https://www.aptagen.com/aptamer-details/?id='

In [203]:
def saveHTML(url):
    root = '/Users/jiamingxu/Downloads/Aptahtmls/'
    urlroot = 'https://www.aptagen.com/'
    if not os.path.exists(root):
        os.mkdir(root)
    os.chdir(root)
    
    if not os.path.exists('./images'):
        os.mkdir('./images')
        
    try:
        r = requests.get(url, headers={'user-agent':'Chrome/16.0'}, timeout=30)
        r.raise_for_status
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, 'html.parser')
        filenametitle = re.sub(r'/','-',soup.title.string.split(':')[-1])  # remove slash in title avoiding file creating error.
        htmlfile = root + url.split('=')[-1] + '.' + filenametitle +  '.html'

        if soup.title.string != 'Apta-Index:':
            if not os.path.exists(htmlfile) or os.stat(htmlfile).st_size == 0:
                with open(htmlfile, 'w') as f:
                    f.write(r.text)
                f.close()

            # Save aptamer image        
            for i in soup('img'):
                try:
                    if i['class'] == ['sequence-diagram']:
                        img = requests.get('https://www.aptagen.com'+i['src'], headers={'user-agent':'Chrome/16.0'}, timeout=30)
                        with open('./images/'+ url.split('=')[-1] + '.' +i['src'].split('.')[-1], 'wb') as imgf:
                            imgf.write(img.content)

                except:
                    continue

        
    except:
        print("ERROR"+str(url.split('=')[-1]))

    
def main():
    url = 'https://www.aptagen.com/aptamer-details/?id='
    for num in range(7250,7350):
        saveHTML(url+str(num))
        time.sleep(1)
    print('Done')
        
main()


Done


In [204]:
def getHTMLText(url):
    try:
        r = requests.get(url, headers={'user-agent':'Chrome/16.0'}, timeout=30)
        r.raise_for_status
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""
    
def parsePage(df, my_dict, html):
    imgroot = '/Users/jiamingxu/Downloads/Aptahtmls/images/'
    soup = BeautifulSoup(open(html), 'html.parser')
    texts = open(html).read()
    aptapat = re.compile(r'[r|f|d][A|G|C|T|U]p')
    aptaseq = re.findall(aptapat, texts) 
    index = html.split('.')[0]
    
    for label in soup('label'):
        if label.string:
            my_dict[label.string] = label.next_sibling.next_sibling.string
    my_dict['Title:'] = soup.title.string.split(':')[1]
    my_dict['Reference:'] = soup.find('h3',{'itemprop':'description'}).string
    my_dict['Sequence:'] = ",".join(aptaseq)
    # Searching for image under /images folder with specific pattern.
    my_dict['Image'] = imgroot + re.search(r''+re.escape(str(index))+'.\w*', ' '.join(os.listdir(imgroot))).group(0)
    df = df.append(my_dict, ignore_index=True)
    return df
    
    
def printAptaSeq(ilt):
    tplt = '{:^30}\t{:^40}'
    print(tplt.format('Name','Sequence'))
    for apta in ilt:
        print(tplt.format(apta[0], apta[1]))
    

def main():
    # Define diction for pandas.
    my_dict = {'Title:':'',
               'Aptamer Chemistry:':'', 
               'Image':'',
               'Target:':'', 
               'Antigen/Target Category:':'', 
               'Affinity (Kd):':'', 
               'Binding Conditions/Buffer:':'', 
               'Binding Temp:':'', 
               'Refolding Program:':'', 
               'Specificity:':'', 
               'Molecular Weight:':'',
               'GC Content:':'',
               'Extinction Coefficient:':'',
               'Ki:':'',
               'LOD:':'',
               'Comments:':'', 
               'Length:':'', 
               'Reference:':'',
               'Sequence:':''} 
    
    # Initialize pandas dataframe
    df = pd.DataFrame(data=[], columns=list(my_dict))

    for file in os.scandir():
        if file.is_file:
            try:
                df = parsePage(df, my_dict, re.search(r'html$',file.name).string)
            except:
                pass
            #parsePage(aptainfo, file.name)
    return df

df = main()
df = df

In [205]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="120" >'

df.to_html('webpage.html', escape=False, formatters=dict(Image=path_to_image_html)) # Add html tag for images

In [207]:
df.to_csv('AptamerDatabase.csv')